# Video Game Console Analysis

## 1. Introduction

## 2. Import Data

In [2]:
# Import libraries 

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import math

## 3. Data Preparation

In [3]:
# Import the data

filepath = './data/games.csv'
column_names = ['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating']

try:
    games_df = pd.read_csv(filepath, header=0, names=column_names)
except FileNotFoundError as e:
    print(f"The file at [{filepath}] was not found. The following error: {e} occurred.\nTry again!")
else:
    print(f"The file at the filepath: [{filepath}] was imported.")

The file at the filepath: [./data/games.csv] was imported.


In [4]:
# Initial Observations

games_df.info()
games_df.head(30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


**Observations**

The dataset comprises 11 columns, each providing specific information about various video games. Upon initial inspection, it becomes apparent that some columns have data types inconsistent with their descriptions.

- The `year_of_release` column is presently of type `float64` and should be converted to type `int64`.
- The `user_score` column is currently of object data type; it would be more appropriate to convert it to a numerical data type to facilitate calculations.

Regarding missing values, several columns exhibit incomplete data: `name`, `year_of_release`, `genre`, `critic_score`, `user_score`, `rating`.

For `name`, `genre`, and `rating`, missing values will be addressed using by either dropping or replacing the values:
- Rows with missing values in the `name` and `genre` columns will be removed, affecting only a small fraction of the dataset (approximately 1 to 3 records), thus preserving data integrity. 
- For the categorical `rating` column, missing values will be filled with the string `'unknown'`. Since these columns primarily serve to categorize data, introducing an `'unknown'` category for each will minimally disrupt the dataset. Removing these values would significantly impact dataset integrity due to their prevalence.

For the columns `critic_score`, `user_score`, and `year_of_release`, missing values will be addressed using dispersion methods such as mean or median:
- The `year_of_release` column will be filled with the median value for that column. This approach, impacting approximately 200 records, maintains data integrity. However, for a larger number of missing values, it might pose challenges.
- Missing `user_score` values will be replaced with the mean for that column. Since the value is somewhere between 1 and 10, not involving zero, the mean will be relatively stable and can be used to fill missing values in that column.
- For `critic_score`, missing values will be grouped by `genre` and `rating` and replaced with the median for those groups, providing a more contextualized approach to handling missing data. Any remaining missing values will be grouped by the `year_of_release` and filled with the median as well.


In [5]:
# handle name and genre missing values
games_df.dropna(subset=['name', 'genre'], inplace=True)

# handle rating missing values
games_df['rating'] = games_df['rating'].fillna('unknown')

# handle year_of_release missing values and data conversion
games_df['year_of_release'].fillna(games_df['year_of_release'].median(), inplace=True)
games_df['year_of_release'] = games_df['year_of_release'].astype('int')

# handle critic_score missing values
games_df['critic_score'] = games_df['critic_score'].fillna(games_df.groupby(['genre', 'rating'])['critic_score'].transform('median'))
games_df['critic_score'].fillna(games_df['critic_score'].median(), inplace=True)


# handle user_score missing values and data conversion
games_df['user_score'] = pd.to_numeric(games_df['user_score'], errors='coerce')
games_df['user_score'] = round(games_df['user_score'].fillna(games_df['user_score'].mean()), 2)

In [6]:
# feature engineering

games_df.insert(8, 'total_sales', games_df['na_sales'] + games_df['jp_sales'] + games_df['eu_sales'] + games_df['other_sales'])
games_df.insert(0, 'game_id', games_df.index)

In [7]:
# Check for duplicates in the dataset

games_df.duplicated().sum()

0

In [8]:
# Re-observe data -- ensure everything is correct

games_df.info()
games_df.head(30)

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16714
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   game_id          16713 non-null  int64  
 1   name             16713 non-null  object 
 2   platform         16713 non-null  object 
 3   year_of_release  16713 non-null  int64  
 4   genre            16713 non-null  object 
 5   na_sales         16713 non-null  float64
 6   eu_sales         16713 non-null  float64
 7   jp_sales         16713 non-null  float64
 8   other_sales      16713 non-null  float64
 9   total_sales      16713 non-null  float64
 10  critic_score     16713 non-null  float64
 11  user_score       16713 non-null  float64
 12  rating           16713 non-null  object 
dtypes: float64(7), int64(2), object(4)
memory usage: 1.8+ MB


,game_id,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,total_sales,critic_score,user_score,rating
0,0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,82.54,76.0,8.00,E
1,1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,40.24,70.0,7.13,unknown
2,2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,35.52,82.0,8.30,E
3,3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,32.77,80.0,8.00,E
4,4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,31.38,75.0,7.13,unknown
5,5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,30.26,68.0,7.13,unknown
6,6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,29.80,89.0,8.50,E
7,7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,28.91,58.0,6.60,E
8,8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,28.32,87.0,8.40,E
9,9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,28.31,67.0,7.13,unknown
